# **InceptionNet - Flower(17) Type Prediction**
---
---

In [1]:
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 47.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0

In [2]:
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=28023d84e1cc1e956f0fb0eb7561164bf1fd57917f0f485479c71ccf8d7fa009
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


## **Get Data**

In [1]:
import tflearn.datasets.oxflower17 as oxflower17
from keras.utils import to_categorical

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
oxflower17

<module 'tflearn.datasets.oxflower17' from '/usr/local/lib/python3.10/dist-packages/tflearn/datasets/oxflower17.py'>

In [3]:
X, y = oxflower17.load_data()

In [4]:
X_train = X.astype("float32")/255.0
y_train = to_categorical(y,   num_classes = 17)

In [5]:
# Check Shape

print(X_train.shape)
print(y_train.shape)

(1360, 224, 224, 3)
(1360, 17)


### **Define Inception Block**

In [6]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense, AveragePooling2D
from keras.optimizers import Adam

In [7]:
from numpy.lib.function_base import kaiser

def inception_block(x, filters):

  tower1 = Conv2D(filters[0], kernel_size=(1,1), padding = "same", activation= 'relu')(x)
  tower1 = Conv2D(filters[1], kernel_size=(3,3), padding = "same", activation= 'relu')(tower1)

  tower2 = Conv2D(filters[2], kernel_size=(1,1), padding = "same", activation= 'relu')(x)
  tower2 = Conv2D(filters[3], kernel_size=(5,5), padding = "same", activation= 'relu')(tower2)

  tower3 = MaxPooling2D(pool_size=(3,3), strides = (1,1),  padding = "same")(x)
  tower3 = Conv2D(filters[4], kernel_size=(1,1), padding = "same", activation= 'relu')(tower3)

  output = concatenate([tower1, tower2, tower3], axis = 3)
  return output


## **Building Inception Model**

In [8]:
def inception(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = inception_block(x, filters=[64, 96, 128, 16, 32])
    x = inception_block(x, filters=[128, 128, 192, 32, 96])
    x = MaxPooling2D((2, 2))(x)

    x = inception_block(x, filters=[192, 96, 208, 16, 48])
    x = inception_block(x, filters=[160, 112, 224, 24, 64])
    x = inception_block(x, filters=[128, 128, 256, 24, 64])
    x = inception_block(x, filters=[112, 144, 288, 32, 64])
    x = MaxPooling2D((2, 2))(x)

    x = inception_block(x, filters=[256, 160, 320, 32, 128])
    x = inception_block(x, filters=[256, 160, 320, 32, 128])
    x = inception_block(x, filters=[384, 192, 384, 48, 128])

    x = AveragePooling2D((4, 4))(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
# Create the Inception model
model = inception(input_shape=(224, 224, 3), num_classes=17)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
model.summary()

# Train
model.fit(X_train, y_train, batch_size=64, epochs=5, verbose=1,validation_split=0.2, shuffle=True)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 64  0           ['conv2d[0][0]']                 
                                )                                                             

/usr/local/lib/python3.10/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 conv2d_4 (Conv2D)              (None, 112, 112, 16  51216       ['conv2d_3[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_5 (Conv2D)              (None, 112, 112, 32  2080        ['max_pooling2d_1[0][0]']        
                                )                                                                 
                                                                                                  
 concatenate (Concatenate)      (None, 112, 112, 14  0           ['conv2d_2[0][0]',               
                                4)                                'conv2d_4[0][0]',               
                                                                  'conv2d_5[0][0]']               
                                                                                                  
 conv2d_6 